In [1]:
%pip install langchain-core langgraph>0.2.27
%pip install -qU "langchain[mistralai]"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import getpass
import os

if not os.environ.get("MISTRAL_API_KEY"):
  os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter API key for Mistral AI: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("mistral-large-latest", model_provider="mistralai")

In [3]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

In [4]:
# Async function for node:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}


# Define graph as before:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

In [10]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "abc123"}}

query = "Hi! I'm Bob. Whenever call me, just call me by my name. Whenever I ask a question, start with my name followed by a comma and then give answer. "

input_messages = [HumanMessage(query)]

# Async invocation:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hi Bob! I'll make sure to address you by your name followed by a comma. Let's start whenever you're ready with your questions.


In [11]:
query = "What is my name?"

input_messages = [HumanMessage(query)]

# Async invocation:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Bob, your name is Bob.


In [ ]:
query = "What is the tallest mountain in Sri Lanka?"

input_messages = [HumanMessage(query)]

# Async invocation:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Bob, the tallest mountain in Sri Lanka is Pidurutalagala, also known as Mount Pedro. It stands at approximately 2,524 meters (8,281 feet) tall.
